#### Forecast Demand whit ARIMA Model

In [1]:
# Importing Basic libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import re
# Importing time series specific libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from scipy.stats import bartlett
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima.model import ARIMA
import pmdarima
from pmdarima import auto_arima
from statsmodels.tsa.statespace import sarimax
import prophet
from prophet import Prophet
from scipy.stats import levene
# Miscellaneous libararies
import warnings
warnings.filterwarnings('ignore')
from math import sqrt
from random import random

# Libaraies for evaluation of model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error
from statsmodels.tsa.arima.model import ARIMAResults


c:\Users\milla\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
 #?  Function for solver data import
def import_data(x):
    """Import the data from a file into the Excel document"""
    global df 
    route_script = os.getcwd()
# Excel name :
    name_doc = x
#Combine script route and name_doc
    route_doc=os.path.join(route_script,name_doc)
#Load data in a Data Frame to panda 
    df=pd.read_excel(route_script)
    return df

In [3]:
 #? call to import data function 
import_data('load_IJV.xlsx')

PermissionError: [Errno 13] Permission denied: 'c:\\Users\\milla\\Documents\\Maestria (Tesis)\\Capitulo 2\\Modelo Arima de la Demanda\\Investigation_Code'